In [3]:
import csv
data = {}
with open('facial_expressions/data/legend.csv') as f:
  reader = csv.reader(f)
  next(reader)
  for row in reader:
    key = row[2].lower()
    if key in data:
      data[key].append(row[1])
    else:
      data[key] = [row[1]]

In [4]:
emotion_list = list(data.keys())
emotion_list

['anger',
 'surprise',
 'disgust',
 'fear',
 'neutral',
 'happiness',
 'sadness',
 'contempt']

In [5]:
# import os

# os.mkdir('master_data')
# os.mkdir('master_data/training')
# os.mkdir('master_data/testing')

In [6]:
# for emotion in emotion_list:
#   os.mkdir(os.path.join('master_data/training/', emotion))
#   os.mkdir(os.path.join('master_data/testing/', emotion))

In [7]:
# from shutil import copyfile
# split_size = 0.8

# for emotion, images in data.items():
#   train_size = int(split_size*len(images))
#   train_images = images[:train_size]
#   test_images = images[train_size:]
#   for image in train_images:
#     source = os.path.join('facial_expressions/images', image)
#     dest = os.path.join('master_data/training', emotion, image)
#     copyfile(source, dest)
#   for image in test_images:
#     source = os.path.join('facial_expressions/images', image)
#     dest = os.path.join('master_data/testing', emotion, image)
#     copyfile(source, dest)

In [8]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten

In [9]:
model = tf.keras.models.Sequential([
      Conv2D(16, (3,3), activation='relu', input_shape = (100, 100, 3)),
      MaxPooling2D(2, 2),
      Conv2D(32, (3,3), activation='relu'),
      MaxPooling2D(2, 2),
      Conv2D(64, (3,3), activation='relu'),
      MaxPooling2D(2, 2),
      Flatten(),
      Dense(1024, activation='relu'),
      Dense(8, activation='softmax')
])
model.compile(optimizer = Adam(lr = 0.01), loss = 'categorical_crossentropy', metrics = ['acc'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 98, 98, 16)        448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 49, 49, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 47, 47, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 23, 23, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 21, 21, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 10, 10, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 6400)              0

C:\Users\NITHISH\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\optimizer_v2\optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


In [10]:
train_dir = 'master_data/training'
test_dir = 'master_data/testing'

train_datagen = ImageDataGenerator(rescale = 1.0/255)
train_generator = train_datagen.flow_from_directory(
                                                    train_dir,
                                                    target_size = (100, 100),
                                                    class_mode = 'categorical',
                                                    batch_size = 128
                                                  )

test_datagen = ImageDataGenerator(rescale = 1.0/255)
test_generator = test_datagen.flow_from_directory(
                                                    test_dir,
                                                    target_size = (100, 100),
                                                    class_mode = 'categorical',
                                                    batch_size = 128
                                                  )

Found 10941 images belonging to 8 classes.
Found 2742 images belonging to 8 classes.


In [11]:
es = EarlyStopping(monitor='val_acc', patience = 2, min_delta=0.01)

In [12]:
model.fit_generator(train_generator,
                    epochs = 10,
                    verbose = 1,
                    validation_data = test_generator,
                    callbacks = [es])

C:\Users\NITHISH\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
86/86 [==============================] - 124s 1s/step - loss: 1.4772 - acc: 0.4763 - val_loss: 1.0147 - val_acc: 0.4362
Epoch 2/10
86/86 [==============================] - 55s 642ms/step - loss: 0.9541 - acc: 0.5903 - val_loss: 1.1271 - val_acc: 0.4336
Epoch 3/10
86/86 [==============================] - 81s 940ms/step - loss: 0.8294 - acc: 0.6742 - val_loss: 0.9688 - val_acc: 0.5613
Epoch 4/10
86/86 [==============================] - 60s 693ms/step - loss: 0.8160 - acc: 0.6951 - val_loss: 0.9268 - val_acc: 0.6488
Epoch 5/10
86/86 [==============================] - 54s 631ms/step - loss: 0.7480 - acc: 0.7306 - val_loss: 0.9825 - val_acc: 0.6123
Epoch 6/10
86/86 [==============================] - 52s 604ms/step - loss: 0.7160 - acc: 0.7440 - val_loss: 0.9946 - val_acc: 0.6036


In [13]:
import numpy as np
y_pred=model.predict(test_generator)

for i in range(len(y_pred)):
  print(emotion_list[np.argmax(y_pred[i])])

neutral
happiness
happiness
happiness
disgust
happiness
happiness
neutral
disgust
happiness
happiness
happiness
disgust
disgust
neutral
neutral
happiness
neutral
happiness
happiness
happiness
disgust
happiness
happiness
happiness
happiness
happiness
neutral
neutral
happiness
neutral
disgust
neutral
happiness
happiness
neutral
happiness
neutral
disgust
disgust
neutral
neutral
neutral
happiness
happiness
happiness
happiness
happiness
happiness
neutral
happiness
happiness
happiness
disgust
neutral
neutral
happiness
happiness
neutral
happiness
neutral
neutral
happiness
disgust
neutral
disgust
happiness
happiness
happiness
happiness
happiness
happiness
happiness
disgust
neutral
happiness
neutral
happiness
neutral
neutral
neutral
disgust
happiness
happiness
happiness
disgust
happiness
disgust
disgust
happiness
neutral
happiness
neutral
disgust
happiness
neutral
disgust
neutral
neutral
contempt
happiness
disgust
happiness
happiness
neutral
happiness
neutral
happiness
happiness
happiness
neutr

In [14]:
results = model.evaluate(test_generator, batch_size=128)
print("test loss, test acc:", results)

22/22 [==============================] - 4s 193ms/step - loss: 0.9946 - acc: 0.6036
test loss, test acc: [0.9945834875106812, 0.6035740375518799]
